In [1]:
import pandas as pd

In [48]:
data = pd.read_csv('transaction_data.csv')
data.head()

,TABLENAME,DM_USERID,DM_SERVICEID,DM_TRANSACTIONID,DM_SERVERTIMESTAMP,DM_APPTIMESTAMP,DM_ACTIVITYTYPE,DM_CLASSKEY,PREIMAGE,POSTIMAGE
0,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:15:000208,2019-02-19:12:06:12:000635,MODIFY,:GSGPM:1900670002:BP0000117:1:1:751717,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...
1,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:13:000967,2019-02-19:12:06:12:000635,MODIFY,:G:1900670002:BP0000222:1:1:751717,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...
2,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::480e0cfb:168e0a46d24:55f,2019-02-12:09:36:16:000782,2019-02-12:10:36:14:000598,MODIFY,:G:1900670015:BP0000222:1:1:705800,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...
3,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::480e0cfb:168e0a46d24:55f,2019-02-12:09:36:17:000893,2019-02-12:10:36:14:000598,MODIFY,:GSGPM:1900670015:BP0000121:1:1:705800,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...
4,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.9::76897e05:16da5065017:-62bf,2019-10-07:07:29:17:000968,2019-10-07:09:28:11:000146,MODIFY,:GSGPM:1900813877:BP0000121:1:1:873973,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...


In [49]:
modified_df = data[['PREIMAGE','POSTIMAGE']]
# modified_df = modified_df.head()
modified_df.head()

,PREIMAGE,POSTIMAGE
0,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...
1,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...
2,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...
3,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...
4,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...


In [50]:
def extract_event_details_from_preimage(preimage_string):
    parts = preimage_string.split('|')
    for part in parts:
        if ' - ' in part and part.count(' - ') == 3:
            try:
                event_type, custodian, country, payment_type = part.split(' - ')
                return pd.Series({
                    "EVENT_TYPE": event_type,
                    "CUSTODIAN": custodian,
                    "COUNTRY": country,
                    "PAYMENT_TYPE": payment_type
                })
            except ValueError:
                continue  # in case split fails
    return pd.Series({
        "EVENT_TYPE": None,
        "CUSTODIAN": None,
        "COUNTRY": None,
        "PAYMENT_TYPE": None
    })
event_info_df = modified_df['PREIMAGE'].apply(extract_event_details_from_preimage)
modified_df2 = pd.concat([modified_df, event_info_df], axis=1)
modified_df2

,PREIMAGE,POSTIMAGE,EVENT_TYPE,CUSTODIAN,COUNTRY,PAYMENT_TYPE
0,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,REDM,SGSS,NL,INCPYMNT
1,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,REDM,SGSS,NL,INCPYMNT
2,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,REDM,SGSS,FR,INCPYMNT
3,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...,GSGPM|1900670015|BP0000121|1|1|705800|1|CR|635...,REDM,SGSS,FR,INCPYMNT
4,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...,GSGPM|1900813877|BP0000121|1|1|873973|1|CR|635...,REDM,SGSS,FR,INCPYMNT
...,...,...,...,...,...,...
494,G|2001125352|BP0000222|2|1|50849|1|DR|635|AMNT...,G|2001125352|BP0000222|2|1|50849|1|DR|635|AMNT...,EXWA,SGSS,FR,VOLU
495,GSGPM|2001125352|BP0000121|2|1|50849|1|DR|635|...,GSGPM|2001125352|BP0000121|2|1|50849|1|DR|635|...,EXWA,SGSS,FR,VOLU
496,G|2001121192|BP0000222|1|1|270430|1|CR|635|AMN...,G|2001121192|BP0000222|1|1|270430|1|CR|635|AMN...,TNDP,SGSS,CH,INCPYMNT
497,GSGPM|2001121192|BP0000145|1|1|270430|1|CR|635...,GSGPM|2001121192|BP0000145|1|1|270430|1|CR|635...,TNDP,SGSS,CH,INCPYMNT


In [51]:
set(modified_df2['PAYMENT_TYPE'])

{'CHOS', 'INCPYMNT', 'MAND', None, 'VOLU'}

In [52]:
# START

In [56]:
import pandas as pd

# Load data
df = pd.read_csv("transaction_data.csv")

def parse_image(image_str):
    parts = image_str.split('|')
    return {i: val for i, val in enumerate(parts)}

# Apply parsing to both PREIMAGE and POSTIMAGE
df['PRE_PARSED'] = df['PREIMAGE'].apply(parse_image)
df['POST_PARSED'] = df['POSTIMAGE'].apply(parse_image)
df.head(3)

,TABLENAME,DM_USERID,DM_SERVICEID,DM_TRANSACTIONID,DM_SERVERTIMESTAMP,DM_APPTIMESTAMP,DM_ACTIVITYTYPE,DM_CLASSKEY,PREIMAGE,POSTIMAGE,PRE_PARSED,POST_PARSED
0,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:15:000208,2019-02-19:12:06:12:000635,MODIFY,:GSGPM:1900670002:BP0000117:1:1:751717,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,"{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ...","{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ..."
1,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:13:000967,2019-02-19:12:06:12:000635,MODIFY,:G:1900670002:BP0000222:1:1:751717,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,"{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '...","{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '..."
2,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::480e0cfb:168e0a46d24:55f,2019-02-12:09:36:16:000782,2019-02-12:10:36:14:000598,MODIFY,:G:1900670015:BP0000222:1:1:705800,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,G|1900670015|BP0000222|1|1|705800|1|CR|635|AMN...,"{0: 'G', 1: '1900670015', 2: 'BP0000222', 3: '...","{0: 'G', 1: '1900670015', 2: 'BP0000222', 3: '..."


In [57]:
def get_diff(pre_dict, post_dict):
    diffs = []
    for k in pre_dict:
        pre_val = pre_dict.get(k, '')
        post_val = post_dict.get(k, '')
        if pre_val != post_val:
            diffs.append((k, pre_val, post_val))
    return diffs

df['DIFFERENCES'] = df.apply(lambda row: get_diff(row['PRE_PARSED'], row['POST_PARSED']), axis=1)
df.head(2)

,TABLENAME,DM_USERID,DM_SERVICEID,DM_TRANSACTIONID,DM_SERVERTIMESTAMP,DM_APPTIMESTAMP,DM_ACTIVITYTYPE,DM_CLASSKEY,PREIMAGE,POSTIMAGE,PRE_PARSED,POST_PARSED,DIFFERENCES
0,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:15:000208,2019-02-19:12:06:12:000635,MODIFY,:GSGPM:1900670002:BP0000117:1:1:751717,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,"{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ...","{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ...","[(29, TOBEPAID, CANC), (145, 2, 3)]"
1,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:13:000967,2019-02-19:12:06:12:000635,MODIFY,:G:1900670002:BP0000222:1:1:751717,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,"{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '...","{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '...","[(29, TOBEPAID, CANC), (145, 2, 3)]"


In [58]:
df['DIFFERENCES'][0]

[(29, 'TOBEPAID', 'CANC'), (145, '2', '3')]

In [59]:
# Example: assuming 'DEPOT ID' is at position 2
def extract_depot_id(image_str):
    return image_str.split('|')[2]  # Adjust index as per actual schema

df['DEPOT_ID'] = df['PREIMAGE'].apply(extract_depot_id)
df.head(2)

,TABLENAME,DM_USERID,DM_SERVICEID,DM_TRANSACTIONID,DM_SERVERTIMESTAMP,DM_APPTIMESTAMP,DM_ACTIVITYTYPE,DM_CLASSKEY,PREIMAGE,POSTIMAGE,PRE_PARSED,POST_PARSED,DIFFERENCES,DEPOT_ID
0,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:15:000208,2019-02-19:12:06:12:000635,MODIFY,:GSGPM:1900670002:BP0000117:1:1:751717,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,GSGPM|1900670002|BP0000117|1|1|751717|1|CR|635...,"{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ...","{0: 'GSGPM', 1: '1900670002', 2: 'BP0000117', ...","[(29, TOBEPAID, CANC), (145, 2, 3)]",BP0000117
1,DCP_DPOT_CSH_PYUT,1,30834,192.208.139.10::51f5081b:1690547b629:-5b8a,2019-02-19:11:06:13:000967,2019-02-19:12:06:12:000635,MODIFY,:G:1900670002:BP0000222:1:1:751717,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,G|1900670002|BP0000222|1|1|751717|1|CR|635|PER...,"{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '...","{0: 'G', 1: '1900670002', 2: 'BP0000222', 3: '...","[(29, TOBEPAID, CANC), (145, 2, 3)]",BP0000222


In [43]:
# One row per change
records = []

for _, row in df.iterrows():
    for diff in row['DIFFERENCES']:
        index, pre, post = diff
        records.append({
            'DEPOT_ID': row['DEPOT_ID'],
            'FIELD_INDEX': index,
            'PRE_VALUE': pre,
            'POST_VALUE': post
        })

diff_df = pd.DataFrame(records)
diff_df

,DEPOT_ID,FIELD_INDEX,PRE_VALUE,POST_VALUE
0,BP0000117,29,TOBEPAID,CANC
1,BP0000117,145,2,3
2,BP0000222,29,TOBEPAID,CANC
3,BP0000222,145,2,3
4,BP0000222,29,TOBEPAID,CANC
...,...,...,...,...
939,BP0000222,168,2,3
940,BP0000145,29,TOBEPAID,CANC
941,BP0000145,168,2,3
942,BP0000222,29,TOBEPAID,CANC


In [61]:
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.preprocessing import LabelEncoder

# # Step 1: Load data
# df = pd.read_csv("transaction_data.csv")

# # Step 2: Parse PREIMAGE and POSTIMAGE into positional dictionaries
# def parse_image(image_str):
#     parts = image_str.split('|')
#     return {i: val for i, val in enumerate(parts)}

# df['PRE_PARSED'] = df['PREIMAGE'].apply(parse_image)
# df['POST_PARSED'] = df['POSTIMAGE'].apply(parse_image)

# # Step 3: Extract differences
# def get_diff(pre_dict, post_dict):
#     diffs = []
#     for k in pre_dict:
#         pre_val = pre_dict.get(k, '')
#         post_val = post_dict.get(k, '')
#         if pre_val != post_val:
#             diffs.append((k, pre_val, post_val))
#     return diffs

# df['DIFFERENCES'] = df.apply(lambda row: get_diff(row['PRE_PARSED'], row['POST_PARSED']), axis=1)

# # Step 4: Extract DEPOT ID (adjust index 2 if needed)
# def extract_depot_id(image_str):
#     parts = image_str.split('|')
#     return parts[2] if len(parts) > 2 else None

# df['DEPOT_ID'] = df['PREIMAGE'].apply(extract_depot_id)

# # Step 5: Flatten differences into a new DataFrame
# records = []
# for _, row in df.iterrows():
#     for diff in row['DIFFERENCES']:
#         index, pre, post = diff
#         records.append({
#             'DEPOT_ID': row['DEPOT_ID'],
#             'FIELD_INDEX': index,
#             'PRE_VALUE': pre,
#             'POST_VALUE': post
#         })

# diff_df = pd.DataFrame(records)

# Step 6: Encode categorical variables
label_encoders = {}
for col in ['DEPOT_ID', 'PRE_VALUE', 'POST_VALUE']:
    le = LabelEncoder()
    diff_df[col] = le.fit_transform(diff_df[col].astype(str))
    label_encoders[col] = le

# FIELD_INDEX is already numerical
X = diff_df[['DEPOT_ID', 'PRE_VALUE']]
y = diff_df['POST_VALUE']

# Step 7: Filter out rare classes (needed for stratify)
value_counts = y.value_counts()
valid_classes = value_counts[value_counts >= 2].index
mask = y.isin(valid_classes)

X_filtered = X[mask]
y_filtered = y[mask]

# Step 8: Train/test split (now with valid stratification)
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, stratify=y_filtered, random_state=42
)

# Step 9: Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)

print("✅ Model trained successfully.")


✅ Model trained successfully.


In [68]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

# Step 1: Load data
df = pd.read_csv("transaction_data.csv")

# Step 2: Parse PREIMAGE and POSTIMAGE into positional dictionaries
def parse_image(image_str):
    parts = image_str.split('|')
    return {i: val for i, val in enumerate(parts)}

df['PRE_PARSED'] = df['PREIMAGE'].apply(parse_image)
df['POST_PARSED'] = df['POSTIMAGE'].apply(parse_image)

# Step 3: Extract differences
def get_diff(pre_dict, post_dict):
    diffs = []
    for k in pre_dict:
        pre_val = pre_dict.get(k, '')
        post_val = post_dict.get(k, '')
        if pre_val != post_val:
            diffs.append((k, pre_val, post_val))
    return diffs

df['DIFFERENCES'] = df.apply(lambda row: get_diff(row['PRE_PARSED'], row['POST_PARSED']), axis=1)

# Step 4: Extract DEPOT ID
def extract_depot_id(image_str):
    parts = image_str.split('|')
    return parts[2] if len(parts) > 2 else None

df['DEPOT_ID'] = df['PREIMAGE'].apply(extract_depot_id)

# Step 5: Flatten differences into a new DataFrame
records = []
for _, row in df.iterrows():
    for diff in row['DIFFERENCES']:
        index, pre, post = diff
        records.append({
            'DEPOT_ID': row['DEPOT_ID'],
            'FIELD_INDEX': index,
            'PRE_VALUE': pre,
            'POST_VALUE': post
        })

diff_df = pd.DataFrame(records)

# Step 6: Encode categorical variables
label_encoders = {}
for col in ['DEPOT_ID', 'PRE_VALUE', 'POST_VALUE']:
    le = LabelEncoder()
    diff_df[col] = le.fit_transform(diff_df[col].astype(str))
    label_encoders[col] = le

# FIELD_INDEX is already numerical
X = diff_df[['DEPOT_ID', 'PRE_VALUE']]
y = diff_df['POST_VALUE']

# Step 7: Filter out rare classes (needed for stratify)
value_counts = y.value_counts()
valid_classes = value_counts[value_counts >= 2].index
mask = y.isin(valid_classes)

X_filtered = X[mask]
y_filtered = y[mask]

# Step 8: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, test_size=0.2, stratify=y_filtered, random_state=42
)

# Step 9: Train the model
model = RandomForestClassifier()
model.fit(X_train, y_train)
print("✅ Model trained successfully.")

# Step 10: User Input and Detailed Prediction Summary
user_input_depot = input("Enter DEPOT ID: ")

# Check if DEPOT ID exists in the label encoder
if user_input_depot not in label_encoders['DEPOT_ID'].classes_:
    print("❌ This DEPOT ID was not seen during training.")
else:
    encoded_depot = label_encoders['DEPOT_ID'].transform([user_input_depot])[0]

    # Get all changes related to this DEPOT_ID
    depot_changes = diff_df[diff_df['DEPOT_ID'] == encoded_depot].copy()

    # Decode labels for readability
    depot_changes['DEPOT_ID'] = label_encoders['DEPOT_ID'].inverse_transform(depot_changes['DEPOT_ID'])
    depot_changes['PRE_VALUE'] = label_encoders['PRE_VALUE'].inverse_transform(depot_changes['PRE_VALUE'])
    depot_changes['POST_VALUE'] = label_encoders['POST_VALUE'].inverse_transform(depot_changes['POST_VALUE'])

    # Display all actual changes
    print(f"\n🔍 Historical changes for DEPOT ID: {user_input_depot}")
    print(depot_changes[['FIELD_INDEX', 'PRE_VALUE', 'POST_VALUE']])

    # Count most frequent field changes
    field_change_counts = depot_changes['FIELD_INDEX'].value_counts().reset_index()
    field_change_counts.columns = ['FIELD_INDEX', 'CHANGE_COUNT']

    print("\n📊 Field Change Frequency:")
    print(field_change_counts)

    # Top 5 most frequent
    top_fields = field_change_counts.head(5)
    print("\n🔥 Top 5 Most Frequently Changed Fields:")
    print(top_fields)

    # 🗣️ Textual Summary
    print("\n🗣️ Textual Summary:")
    total_changes = len(depot_changes)
    most_frequent_field = top_fields.iloc[0]['FIELD_INDEX']
    most_frequent_count = top_fields.iloc[0]['CHANGE_COUNT']

    print(f"🔸 A total of {total_changes} changes were recorded for DEPOT ID '{user_input_depot}'.")
    print(f"🔸 The most frequently changed field was at position {int(most_frequent_field)}, with {int(most_frequent_count)} changes.")
    print("🔸 Other commonly changed fields include:", ", ".join(str(int(i)) for i in top_fields['FIELD_INDEX'][1:].tolist()))




✅ Model trained successfully.
Enter DEPOT ID: BP0000222

🔍 Historical changes for DEPOT ID: BP0000222
     FIELD_INDEX PRE_VALUE POST_VALUE
2             29  TOBEPAID       CANC
3            145         2          3
4             29  TOBEPAID       CANC
5            145         1          2
12            29  TOBEPAID       CANC
..           ...       ...        ...
935          168         1          2
938           29  TOBEPAID       CANC
939          168         2          3
942           29  TOBEPAID       CANC
943          168         2          3

[222 rows x 3 columns]

📊 Field Change Frequency:
   FIELD_INDEX  CHANGE_COUNT
0           29           111
1          145            54
2          151            25
3          167            21
4          168             8
5          173             3

🔥 Top 5 Most Frequently Changed Fields:
   FIELD_INDEX  CHANGE_COUNT
0           29           111
1          145            54
2          151            25
3          167            21
4 